In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pre = pd.read_excel('EP42_45degree_FFP.xlsx')
next = pd.read_excel('Epix42_PR_45degree.xlsx')

In [5]:
pre.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
119,8837,Airtight_table,130,連續斜率異常_常壓,2/410,0.49
154,17809,Epix_Pro_Baro,1,Calibrate Golden Sample,1/402,0.25
153,17808,Epix_Pro_GSensor,17,Calibration,1/404,0.25


In [7]:
next.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
44,17886,Epix_pro_RTESN,75,Check ECG 1907 IOP,23/629,3.66
203,17822,Epix_Pro_ALS,18,Temperature Sensor Temperature(After),1/562,0.18
119,14545,HRM_lensLifted_Unit,1,RS232連線|RS232 Connection,2/307,0.65


# Tweak and Filter

Blacklist

In [8]:
nextocessTypeBlackList = 'Pack|Click|RabbitCard|EasyCard|DeleteBundle|nextodScan|FileCopyer'
ItemNameBlackList = 'ESN|Check nextoductionMap|Fixture ID'

tweak_filter def

In [9]:
def tweak_filter(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    .query("~ProcessType.str.contains(@nextocessTypeBlackList) and ~ItemName.str.contains(@ItemNameBlackList)")
    # .CountESN.quantile(0.03)
    .query("CountESN > CountESN.quantile(0.05)")
    .query("Retest >= Retest.mean()")
    # .Retest.mean()
    )

filter def

In [10]:
def tweak(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    )

In [11]:
# tweak_filter(next).to_excel('tweak_EP47_45degree_PR.xlsx', index=False)

# Merge two phases

In [12]:
pre_next = pd.merge(
    left=tweak_filter(pre), right=tweak(next), how='left',
    left_on=['ItemNameType','ProcessType','Item','ItemName'], right_on=['ItemNameType','ProcessType','Item','ItemName'],
    suffixes=['_pre','_next'])

In [13]:
pre_next.to_excel('output.xlsx', index=False)

# RetryImprove def

In [14]:
def RetryImprove(df):
    return((100 * (df.Retest_pre.sum() - df.Retest_next.sum()) / df.Retest_pre.sum()).round(3))

In [15]:
RetryImprove(pre_next)

60.718

In [16]:
def RetryDiff(df):
    return(df
     .assign(diff = df.Retest_pre - df.Retest_next)
     [['ProcessType','ItemName','diff']]
     .sort_values('diff', ascending=False)
     .dropna(axis=0)
     .style.bar('diff', color=['red','green'])
     )
RetryDiff(pre_next)

,ProcessType,ItemName,diff
0,Epix_Pro_ECG_FT,ECG impedance short,15.880000
2,Epix_Pro_ATE_TI,AFE Power Consumption data,14.450000
3,Epix_Pro_ATE_TI,Temperature Sensor Temperature,12.820000
5,Epix_Pro_Gyro,電池電壓|Battery Voltage,6.660000
24,Epix_Pro_FT2,Power Box Sampling BL on normal(Avg) diff with ATE,1.750000
11,Epix_Pro_FT1,GPS_ (L1 + L5) by ref,1.270000
18,Epix_Pro_Gyro,Check FT3 Test Data,1.170000
20,Epix_Pro_FT2,ANT_power 2450MHz,0.880000
17,Epix_Pro_ATE_TI,AFE Power Consumption Test,0.670000
15,Epix_Pro_FT1,006J Comparison,-0.300000
